# import libraries

In [158]:
import os
import glob
import pandas as pd

# import and explore data

**import data**

In [159]:
raw_data_path = "/home/phillip/Dokumente/repositories/ds1_weather_data_ger/data/raw_data/raw_data_stations/stations_basedata.txt"

df_data_raw = pd.read_fwf(raw_data_path
 ,widths=[5,9,9,10,6,12,10,20,50]
 ,encoding='ISO-8859-1')
print(df_data_raw)

#widths=[5,9,9,10,6,12,10,20,50]


      Stati  ons_id vo  n_datum b  is_datum S  tation  shoehe geoBr  \
0     -----  ------ --  ------- -  -------- -  ------  ------ -----   
1     00001   19370101   19860630         NaN     478       47.8413   
2     00003   18910101   20110331         NaN     202       50.7827   
3     00011   19800901   20200930         NaN     680       47.9736   
4     00044   19690101   20230613         NaN      44       52.9336   
...     ...        ...        ...         ...     ...           ...   
1346  19607   19720501   20010430         NaN     615       50.6952   
1347  19617   19410101   20061231         NaN     310       49.7391   
1348  19631   19060101   19691231         NaN     268       50.8083   
1349  19647   19510101   20051031         NaN     178       49.4547   
1350  19774   19710819   19940228         NaN     174       51.4814   

      eite geoLa  enge Stationsname Bu                                ndesland  
0     ---- -----  ---- ---------------   -------------------------

**explore data**

In [160]:
#show number of columns and rows
df_data_raw.shape

# show column names and data types
df_data_raw.info() 

df_data_raw.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351 entries, 0 to 1350
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Stati                 1351 non-null   object
 1   ons_id vo             1351 non-null   object
 2   n_datum b             1351 non-null   object
 3   is_datum S            1 non-null      object
 4   tation                1351 non-null   object
 5   shoehe geoBr          1351 non-null   object
 6   eite geoLa            1351 non-null   object
 7   enge Stationsname Bu  1351 non-null   object
 8   ndesland              1351 non-null   object
dtypes: object(9)
memory usage: 95.1+ KB


,Stati,ons_id vo,n_datum b,is_datum S,tation,shoehe geoBr,eite geoLa,enge Stationsname Bu,ndesland
0,-----,------ --,------- -,-------- -,------,------ -----,---- -----,---- ---------------,-------------------------- ----------
1,00001,19370101,19860630,NaN,478,47.8413,8.8493,Aach,Baden-Württemberg
2,00003,18910101,20110331,NaN,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
3,00011,19800901,20200930,NaN,680,47.9736,8.5205,Donaueschingen (Land,eplatz) Baden-Württemberg
4,00044,19690101,20230613,NaN,44,52.9336,8.2370,Großenkneten,Niedersachsen


**findings for the transformation**
- row 0 has senseless values -> delete
- read in of data is not easy, there is no seperator -> manually edit columns
- not all columns and rows are needed -> clear
- no clear datatpyes -> datatypes has to be definied           

# data transformation

**rename columns and delete not needed columns/rows**

In [161]:
#rename columns
df_data_raw.rename(columns={
    'Stati':'station_id',
    'ons_id vo':'date_commissioning', 
    'tation':'heigth_station[m]',
    'shoehe geoBr':'geo_width',
    'eite geoLa':'geo_length'
    }
    ,inplace=True)

#delete columns
columns_to_drop = ['n_datum b','is_datum S','enge Stationsname Bu','ndesland']
df_data_raw = df_data_raw.drop(columns=columns_to_drop)

#delete rows
df_data_raw  = df_data_raw.drop(0)

print(df_data_raw.head())

  station_id date_commissioning heigth_station[m] geo_width geo_length
1      00001           19370101               478   47.8413     8.8493
2      00003           18910101               202   50.7827     6.0941
3      00011           19800901               680   47.9736     8.5205
4      00044           19690101                44   52.9336     8.2370
5      00052           19690101                46   53.6623    10.1990


**edit data types**

In [162]:
df_data_raw['date_commissioning'] = pd.to_datetime(df_data_raw['date_commissioning'], format='%Y%m%d')

df_data_raw['station_id'] = df_data_raw['station_id'].astype(int)
df_data_raw['geo_length'] = df_data_raw['geo_length'].astype(float)
df_data_raw['geo_width'] = df_data_raw['geo_width'].astype(float)
df_data_raw['heigth_station[m]'] = df_data_raw['heigth_station[m]'].astype(int)

print(df_data_raw.dtypes)
print(df_data_raw)

station_id                     int64
date_commissioning    datetime64[ns]
heigth_station[m]              int64
geo_width                    float64
geo_length                   float64
dtype: object
      station_id date_commissioning  heigth_station[m]  geo_width  geo_length
1              1         1937-01-01                478    47.8413      8.8493
2              3         1891-01-01                202    50.7827      6.0941
3             11         1980-09-01                680    47.9736      8.5205
4             44         1969-01-01                 44    52.9336      8.2370
5             52         1969-01-01                 46    53.6623     10.1990
...          ...                ...                ...        ...         ...
1346       19607         1972-05-01                615    50.6952     13.4972
1347       19617         1941-01-01                310    49.7391     10.6039
1348       19631         1906-01-01                268    50.8083     10.2294
1349       19647     

**safe necessary entry**

In [163]:
#necessary:
    # HAMBURG   > station ID = 1975
    # LEIPZIG   > station ID = 2932
    # KIEL      > station ID = 2564
    # KÖLN      > station ID = 2667
    # MÜNCHEN   > station ID = 3379
    # STUTTGART > station ID = 4928

relevant_station_ids = [1975,2932,2564,2667,3379,4928]

# Behalten Sie nur die Zeilen, in denen 'Spalte' einen der Werte in behaelt_werte enthält
df_data_raw = df_data_raw[df_data_raw['station_id'].isin(relevant_station_ids)]

df_data_raw['city'] = df_data_raw['station_id']
df_data_raw['city'] = df_data_raw['city'].replace([1975,2932,2564,2667,3379,4928],['HAMBURG','LEIPZIG','KIEL','KOELN','MUENCHEN','STUTTGART'])

print(df_data_raw)



     station_id date_commissioning  heigth_station[m]  geo_width  geo_length  \
409        1975         1936-01-01                 11    53.6332      9.9881   
531        2564         1974-01-01                 28    54.3776     10.1424   
551        2667         1957-07-01                 92    50.8645      7.1575   
604        2932         1934-01-01                131    51.4347     12.2396   
703        3379         1954-06-01                515    48.1632     11.5429   
985        4928         1958-01-01                314    48.8281      9.2000   

          city  
409    HAMBURG  
531       KIEL  
551      KOELN  
604    LEIPZIG  
703   MUENCHEN  
985  STUTTGART  


# final export data in csv

In [164]:
#OUTPUT PATH
cleared_data_saving_path = "/home/phillip/Dokumente/repositories/ds1_weather_data_ger/data"

#OUTPUT ACTION
df_data_raw.to_csv(os.path.join(cleared_data_saving_path, "station_data_transformed.csv"), index=False, encoding='utf-8-sig')